In [4]:
#Libraries for data manipulation
import pandas as pd
import numpy as np

#Library for splitting data
from sklearn.model_selection import train_test_split 

#Libraries for hyperparameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [3]:
#Searchs for the best hyperparameters using Grid Search
def findHyperparametersGS(x, y, model, param_grid, cv):
    xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.25, random_state = 42, cv = cv)
    grid = GridSearchCV(estimator = model, param_grid = param_grid, scoring = 'roc_auc')
    grid.fit(xTrain, yTrain)
    return grid.best_params_

In [ ]:
#Searchs for the best hyperparameters using Randomized Search
def findHyperparametersRS(x, y, model, param_grid, n_iter):
    xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.25, random_state = 42)
    grid = RandomizedSearchCV(estimator = model, params_grid, n_iter = n_iter, scoring='roc_auc')
    grid.fit(xTrain, yTrain)
    return grid.best_params_

## Random forest

In [ ]:
param_grid = { 
    'n_estimators': [100, 300, 500, 800, 1200],
    'max_depth' : [5, 8, 15, 25, 30],
    'min_samples_split' = [2, 5, 10, 15, 100],
    'min_samples_leaf' = [1, 2, 5, 10]
}

RC = RandomForestClassifier(random_state = 42)
#bestParameters = findHyperparametersGS(trainCSV, labels, RC, param_grid, 5)